# Import Libraries

In [ ]:
# for creating neural network architecture
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

# for image processing
import cv2
import numpy as np

# for dataset management
import os, shutil

# for time management
from tqdm import tqdm

# Setting up hardware

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')

if physical_devices != []:
    print("Using GPU")
    for i in physical_devices:
        tf.config.experimental.set_memory_growth(i, True)
else:
    print("Using CPU")
    pass

# Load Dataset

In [ ]:
root_dir = str(input("Path where 'classify train' directory belongs: "))
classify_train = os.path.join(root_dir, 'classify train')

train_directory = os.path.join(classify_train, 'training')
validation_directory = os.path.join(classify_train, 'validation')
test_directory = os.path.join(classify_train, 'testing')

# Analyze Dataset

### Get image data

In [ ]:
height_lis = []
width_lis = []

total = sum([len(files) for r, d, files in os.walk(classify_train)])

with tqdm(total=total) as pbar:
    for splitname in os.listdir(classify_train):
        split_dir = os.path.join(classify_train, splitname)
        for class_name in os.listdir(split_dir):
            class_dir = os.path.join(split_dir, class_name)
            for filename in os.listdir(class_dir):
                file_path = os.path.join(class_dir, filename)
                img = cv2.imread(file_path)

                height_lis.append(img.shape[0])
                width_lis.append(img.shape[1])

                pbar.set_description("Progress")
                pbar.update()

In [ ]:
avg_height = int(np.mean(height_lis))
avg_width = int(np.mean(width_lis))

smallest_height = int(min(height_lis))
smallest_width = int(min(width_lis))

print('Average height & width respectively: {}, {}'.format(avg_height, avg_width))
print('Smallest height & width respectively: {}, {}'.format(smallest_height, smallest_width))

### Resize Image

In [ ]:
from tqdm import tqdm 

total = sum([len(files) for r, d, files in os.walk(classify_train)])

with tqdm(total=total) as pbar:
    for splitname in os.listdir(classify_train):
        split_dir = os.path.join(classify_train, splitname)
        for class_name in os.listdir(split_dir):
            class_dir = os.path.join(split_dir, class_name)
            for filename in os.listdir(class_dir):
                file_path = os.path.join(class_dir, filename)
                img = cv2.imread(file_path)
                res = cv2.resize(img, (128, 128))
                cv2.imwrite(file_path, res)
            
                pbar.set_description("Progress")
                pbar.update()

# Callbacks

In [ ]:
learning_rate = 0.001
steps = 1 # change steps to 1 to apply exponential decay

def lr_schedule(epoch):
    return learning_rate * (0.1 ** int(epoch / steps))


callback = [tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose = 1),
            tf.keras.callbacks.EarlyStopping(monitor = 'loss', min_delta = 0.001, patience = 10, verbose = 1, mode = "min")]

# Generator

In [ ]:
class_no = len(os.listdir(train_directory))

if class_no <= 2:
    class_mode = 'binary'
    output_activation = 'sigmoid'
    output_neurons = 1
else:
    class_mode = 'categorical'
    output_activation = 'softmax'
    output_neurons = class_no
   
target_size = res.shape[0:2]
dim = res.shape

# Model Architecture

In [ ]:
no_layers = int(input('Conv2d with activation + Max-pool + Dropout for feature extraction = 1 feature extraction layer \nHow many of such feature extraction layers you want to use ? '))    
no_conv = int(input('How many conv2d layers you want to use in each feature extraction layer ? '))
no_filters = int(input('Put no. of filters in 1st conv2d layer: '))
size_filter = int(input('Enter size of filter (width or height): '))
f_dropout = int(input('Enter dropout rate for feature extraction: '))/100

no_d_layers = int(input('Dense with activation + Dropout for desnse layer = 1 dense layer \nHow many of such dense layers you want to use ? '))
d_neurons = int(input('Enter no.of neurons you want to use in 1st dense layer: '))
d_dropout = int(input('Enter dropout rate for dense layer: '))/100

**Don't put user inputs inside the function below as it'll be called multiple times inside a loop**

In [ ]:
def Custom_Model():
    
    model = Sequential(name = 'CUSTOM')
    
    
    # feature extraction
    m, n = 0, 0 # m = increamental factor of no. of filters, # n = total no. of filters in convolution layer
    for l in range(no_layers):
        m = 2**l  
        n = no_filters*m 
        for i in range(no_conv):
            model.add(Conv2D(n, 
                             (size_filter,size_filter), 
                             kernel_regularizer=l2(lambd), 
                             bias_regularizer=l2(lambd),
                             padding = 'same', 
                             input_shape = dim))
            model.add(LeakyReLU())
        model.add(MaxPooling2D(2, 2))
        model.add(Dropout(f_dropout))
    
    
    # flatten
    model.add(Flatten())
    
    
    # dense layer
    m, n = 0, 0
    for d in range(no_d_layers):
        m = 2**d
        n = d_neurons//m
        model.add(Dense(n, kernel_regularizer=l2(lambd), bias_regularizer=l2(lambd)))
        model.add(LeakyReLU())
        model.add(Dropout(d_dropout))
    model.add(Dense(output_neurons, output_activation))

    
    return model

# Research

In [ ]:
epoch = 100

lambd_list = []
lambd_no = int(input('Using how many lambdas you want to train with ? '))
print('Enter {} lambda value/values consecutively:'.format(lambd_no))
for i in range(lambd_no): 
    lambd = float(input('Enter lambda value: '))
    lambd_list.append(lambd)

batch_list = []
batch_no = int(input("Enter how many batches you want to use: "))
print('Enter {} batch no. consecutively:'.format(batch_no))
for j in range(batch_no):
    b_size = int(input('Enter batch size: '))
    batch_list.append(b_size)

In [ ]:
# iterate over batch sizes
for batch_size in batch_list: 
    train_datagen = ImageDataGenerator(rescale=1.0/255.0)
    train_generator = train_datagen.flow_from_directory(train_directory,
                                                        batch_size = batch_size,
                                                        class_mode = class_mode,
                                                        target_size = target_size)

    val_datagen = ImageDataGenerator(rescale=1.0/255.0)
    validation_generator = val_datagen.flow_from_directory(validation_directory,
                                                        batch_size = batch_size,
                                                        class_mode = class_mode,
                                                        target_size = target_size)

    test_datagen = ImageDataGenerator(rescale=1.0/255.0)
    test_generator = test_datagen.flow_from_directory(test_directory,
                                                        batch_size = batch_size,
                                                        class_mode = class_mode,
                                                        target_size = target_size)
    
    # iterate over lambdas
    for lambd in lambd_list:
        model = Custom_Model()
        
        # COMPILE
        loss = 'categorical_crossentropy'
        optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)
        model.compile(loss = loss,
                    optimizer = optimizer,
                    metrics=['accuracy',
                    tf.keras.metrics.TruePositives(), 
                    tf.keras.metrics.TrueNegatives(), 
                    tf.keras.metrics.FalsePositives(),
                    tf.keras.metrics.FalseNegatives()])

        print ('\n************ for lambda = {}************\n'.format(lambd))

        # FIT
        history = model.fit(train_generator,
                        epochs = epoch,
                        verbose = 1,
                        callbacks = callback,
                        validation_data = validation_generator,
                        shuffle = True)

        # PLOT
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        epochs = range(len(acc))
        import matplotlib.pyplot as plt
        
        # Accuracy vs Epochs
        plt.plot(epochs, acc, 'r', label='Training Accuracy')
        plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")
        plt.title('Training and validation accuracy vs Epochs')
        plt.legend()
        plt.show()
        
        # Loss vs Epochs
        plt.plot(epochs, loss, 'r', label="Training Loss")
        plt.plot(epochs, val_loss, 'b', label="Validation Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.title('Training and validation loss vs Epochs')
        plt.legend()
        plt.show()

        # ACCURACIES
        print("Training accuracy: {}".format(model.evaluate(train_generator, verbose=0)[1]))
        print("Validation accuracy: {}".format(model.evaluate(validation_generator, verbose=0)[1]))
        print("Blind test accuracy: {}".format(model.evaluate(test_generator, verbose=0)[1]))
        tp = int(model.evaluate(test_generator, verbose=0)[2])
        tn = int(model.evaluate(test_generator, verbose=0)[3])
        fp = int(model.evaluate(test_generator, verbose=0)[4])
        fn = int(model.evaluate(test_generator, verbose=0)[5])
        sensitivity = (tp/(tp+fn))*100
        specificity = (tn/(tn+fp))*100
        print("Sensitivity: {}".format(sensitivity))
        print("Specificity: {}".format(specificity))